<a href="https://colab.research.google.com/github/vinay18-irpanwar/Google-Colab/blob/main/iterative_reflection_LANGGRAPH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install langchain langchain_community langgraph langchain_google_genai

In [34]:
#LLM
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

LLM=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=userdata.get('gemikey')
)

In [35]:
from typing import TypedDict

class IterReflectstate(TypedDict):
  task:str
  max_iteration:int
  iteration:int
  code:str
  feedback:str

In [36]:
from langgraph.graph import StateGraph,START,END

workflow=StateGraph(IterReflectstate)

In [37]:
#prompts
from langchain_core.prompts import PromptTemplate

generat_prompt=PromptTemplate(
    input_variables=["task"],
    template="""
      You are an expert Python developer.

      Generate Python code to accomplish the following task:
      {task}

      Constraints:
      - Output ONLY valid Python code.
      - Do NOT include explanations, comments, or Markdown formatting.
      - Ensure the code is complete and runnable as-is.
    """
)

reflect_prompt=PromptTemplate(
    input_variables=["code"],
    template="""
      You are an expert Python code reviewer.

      Review the following code thoroughly:
      {code}

      Provide a structured assessment containing:
      1. **Brief critique** of the logic, structure, readability, and overall correctness.
      2. **Specific improvements** with actionable suggestions (performance, clarity, maintainability, robustness).
      3. **Missing test cases or edge cases** that should be considered to ensure reliable behavior.

      Keep the feedback concise, objective, and practical.
    """
)

improve_prompt=PromptTemplate(
    input_variables=["feedback", "code"],
    template="""
      You are an expert Python developer.

      Based on the following feedback from a Senior Python Developer, revise and improve the previously generated code:
      {feedback}

      Requirements:
      - Make the code more efficient, clean, and readable.
      - Ensure it handles all relevant edge cases.
      - Output ONLY the improved Python code (no explanations, no comments, no Markdown).

      Original Code:
      {code}
    """
)


In [38]:
#add node
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
def generate_code(state:IterReflectstate):

  chain=generat_prompt|LLM|StrOutputParser()
  state['code']=chain.invoke({"task":state['task']})
  state['iteration']=0   #state.get("iteration",0)  # <--in state agar iteration mela to -, or nai mela to 0
  return state

def reflect_code(state:IterReflectstate):

  chain=reflect_prompt|LLM|StrOutputParser()
  state['feedback']=chain.invoke({"code":state['code']})
  return state

def improve_code(state:IterReflectstate):

  chain=improve_prompt|LLM|StrOutputParser()
  state['code']=chain.invoke({"feedback":state['feedback'],"code":state['code']})
  state['iteration']=state['iteration']+1
  return state

workflow.add_node("generate_code",generate_code)
workflow.add_node("reflect_no_code",reflect_code)
workflow.add_node("improve_code",improve_code)

In [39]:
#add_edges

def improve(state:IterReflectstate):
  if state['iteration'] < state['max_iteration']:
    return "continue"
  else:
    return "END"


workflow.add_edge(START,"generate_code")
workflow.add_edge("generate_code","reflect_no_code")
workflow.add_edge("reflect_no_code","improve_code")
workflow.add_conditional_edges("improve_code",improve,{
    "continue":"reflect_no_code",
    "END":END
})

In [40]:
app=workflow.compile()

In [48]:
initial_state={"max_iteration":1,"task":"addition of two no. "}

res=app.invoke(initial_state)
print("".join(res['code']))

def add_numbers(a: int | float, b: int | float) -> int | float:
    """
    Adds two numbers and returns their sum.

    Args:
        a (int | float): The first number.
        b (int | float): The second number.

    Returns:
        int | float: The sum of the two numbers.

    Raises:
        TypeError: If either input is not an int or a float.
    """
    if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
        raise TypeError("Both inputs must be numbers (int or float).")
    return a + b

if __name__ == '__main__':
    # Demonstrating valid use cases
    print(add_numbers(5, 7))
    print(add_numbers(2.5, 3.7))
    print(add_numbers(-10, 5))
    print(add_numbers(0, 100))
    print(add_numbers(5, 7.5))

    # Demonstrating error handling for invalid types
    try:
        add_numbers("hello", "world")
    except TypeError as e:
        print(f"Caught expected error: {e}")

    try:
        add_numbers(10, [1, 2])
    except TypeError as e:
        print(f"C